In [ ]:
import WDFunctions as w

take_data_from_url = input('''
Data will be taken from spreadsheets or downloaded from url.
Do you have the spreadsheets available? [Y/N]
''')

if take_data_from_url.lower() == 'y':
    file = 'publishedweek192020.xlsx'
    tabs = w.loadxlstabs(file)
    w.WeeklyDeathsByRegion(tabs)
    w.WeeklyDeathsByAgeSex(tabs)

    file = 'lahbtablesweek19.xlsx'
    reg_data = w.pd.read_excel(file, sheet_name='Registrations - All data', skiprows=3)
    occ_data = w.pd.read_excel(file, sheet_name='Occurrences - All data', skiprows=3)
    w.WeeklyDeathsByLA_HB(reg_data, occ_data, '2020')
    
    print('Transforms complete!')

else:
    print('Taking data from url download')
    
    from gssutils import *

    # get weekly deaths data from url -> by regions & by sex and age
    url = 'https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales'
    scraper = Scraper(url)

    # get required tabs -> passed to functions
    tabs = scraper.distributions[0].as_databaker()

    # get weekly deaths data from url -> by local authority & health board
    url = 'https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/deathregistrationsandoccurrencesbylocalauthorityandhealthboard'
    scraper = Scraper(url)

    # import tabs separately -> registrations & occurrences -> to pass to function
    reg_data = scraper.distributions[0].as_pandas(sheet_name='Registrations - All data', skiprows=3)
    occ_data = scraper.distributions[0].as_pandas(sheet_name='Occurrences - All data', skiprows=3)

    # run functions
    w.WeeklyDeathsByRegion(tabs)
    w.WeeklyDeathsByAgeSex(tabs)
    w.WeeklyDeathsByLA_HB(reg_data, occ_data, '2020')
    
    print('Transforms complete!')